In [ ]:
 !wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-02 16:51:06--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-02 16:51:07--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc64fd4303856e39fe3402977bdb.dl.dropboxusercontent.com/cd/0/inline/BNt8FJyBiLQVM1kuqUFDPl13woeTHJ1hbNJCE43VGKVjQxc_mpSrznme7PFZ_khV3_zySeGwZwj-vIM8TuPgDlXas7s5EYZh-bLl7WsdNcpvYPi-lGdWgRFxCXjEAqfuD7iuJZ1CV3wx85jd5wzcmYp3/file# [following]
--2021-05-02 16:51:07--  https://uc64fd4303856e39fe3402977bdb.dl.dropboxusercontent.com/cd/0/inline/BNt8FJyBiLQVM1kuqUFDPl13woeTHJ1hbNJCE43VGKVjQxc_mpSrznme7PFZ_khV

In [ ]:
!wget https://www.dropbox.com/sh/8lbu926vr9cqgjt/AACv4Ajik2Uxf1eRpsbzZF7Pa?dl=0 -O model.zip
!unzip -o model.zip -d model

--2021-05-02 16:51:35--  https://www.dropbox.com/sh/8lbu926vr9cqgjt/AACv4Ajik2Uxf1eRpsbzZF7Pa?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/8lbu926vr9cqgjt/AACv4Ajik2Uxf1eRpsbzZF7Pa [following]
--2021-05-02 16:51:35--  https://www.dropbox.com/sh/raw/8lbu926vr9cqgjt/AACv4Ajik2Uxf1eRpsbzZF7Pa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfed0e4fbc9af06cfb289619a02.dl.dropboxusercontent.com/zip_download_get/AxJkF-Uan3XBdun0w8J-w77Gnn2i4JwxVNpm1luk3Hs_ZRckIcYIo6KWBkSEHKirdHiCNQhMIAPuT8SxQYu0mwVrla0h81YLdx9dJar8zZV-bw [following]
--2021-05-02 16:51:36--  https://ucfed0e4fbc9af06cfb289619a02.dl.dropboxusercontent.com/zip_download_get/AxJkF-Uan3XBdun0w8J-w77Gnn2i4JwxVNpm1luk3Hs_ZRckIcYIo6KWBkSEHKirdHiCNQhMIAPuT8SxQ

In [ ]:
!pip install transformers
import pickle
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments

In [ ]:
from transformers import pipeline

chef = pipeline('text-generation', model='./model', tokenizer='gpt2', config={'max_length':1500})

chef("How was your day? <BOS>")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "How was your day? <BOS> I've been working (2x a week, maybe more so) for 7 years and today I finally got that job! <EOS> \nDo you guys feel pressured to have more sex?"}]

In [ ]:
with open("grouped_data.pickle", "rb") as f:
  data = pickle.load(f)

In [ ]:
min = 0
max = 10000
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
from tqdm.notebook import tqdm
import re

# from pprint import pprint
# fakes_plus_real = {}
i = 2001
for key in tqdm(list(data.keys())[2001:10000]):
    post = data[key][0]
    
    title = post['title'] + " <BOS>"
    top_comment = data[key][1][0]["body"]
    
    if top_comment != "[deleted]":
      fakes_plus_real[key] = {}
      fakes_plus_real[key]["title"] = title
      fakes_plus_real[key]["real_comment"] = top_comment.replace('\n', " ")

      # generate and clean
      fakes_plus_real[key]["fake_comment"] = chef(title)[0]['generated_text'][len(title) + 1:]
      fakes_plus_real[key]["fake_comment"] = fakes_plus_real[key]["fake_comment"].split('<EOS>')[0]
      fakes_plus_real[key]["fake_comment"] = fakes_plus_real[key]["fake_comment"].replace('\n', " ")
      re.sub(' +', ' ', fakes_plus_real[key]["fake_comment"])

      i += 1

    if i % 1000 == 0:
      with open(f"fakes_plus_real{i}.pickle", "wb") as f:
        pickle.dump(fakes_plus_real, f)
      
      !cp "fakes_plus_real{i}.pickle" "drive/MyDrive/Colab Notebooks/reddit"
    
    

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
with open("fakes_plus_real.pickle", "wb") as f:
  pickle.dump(fakes_plus_real, f)

!cp "fakes_plus_real.pickle" "drive/MyDrive/Colab Notebooks/reddit"

In [ ]:
with open("fakes_plus_real.pickle", "rb") as f:
  temp = pickle.load(f)

temp

{'27lq60': {'fake_comment': '[removed] ',
  'real_comment': "Pretty tame. Woke up early to go to some garage sales around my neighborhood (They do a community garage sale every year.) Then went to Petsmart for the Puppy Adoption event because I'm considering getting a dog, and I just wanted to play with the puppies. Spent the rest of the day playing Battlefront 2 and Civ 5.\n\nOverall, a pretty great day.",
  'title': 'How is your day today? <BOS>'}}